In [1]:
import json
from glob import glob
from os.path import join as pjoin
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt

from match_elements.GUI_pair import GUIPair
from GUI import GUI
from Operation import Operation

ModuleNotFoundError: No module named 'match_elements'

In [2]:
from keras.applications.resnet import ResNet50
resnet_model = ResNet50(include_top=False, input_shape=(32, 32, 3))

94765736/94765736 [==============================] - 3s 0us/step


In [3]:
from paddleocr import PaddleOCR
paddle_ocr = PaddleOCR(use_angle_cls=True, lang='en')

C:\Users\xie029\AppData\Roaming\Python\Python310\site-packages\skimage\util\dtype.py:27: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),


[2023/02/15 16:45:28] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\xie029/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\xie029/.paddleocr/whl\\rec\\en\\en_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, 

In [4]:
# load operations
testing_data_root = '/home/ml/Data/visual testing/experiments/humanoid-output/adguard-1'

steps_file = pjoin(testing_data_root, 'output.json')
steps = json.load(open(steps_file, 'r'))

operations = []
for step in steps:
    op = Operation(pjoin(testing_data_root, step['tag'] + '.jpg'), step['event_type'], step['bounds'])
    op.detect_text(paddle_ocr)
    op.shrink_target_element_img()
    op.resize(int(op.ui_img_width * (800 / op.ui_img_height)), 800)        
    operations.append(op)
#     op.show_target_ele()
#     break
# opr = operations[0]
# opr.show_target_ele()

[2022/03/02 17:16:15] root DEBUG: dt_boxes num : 1, elapse : 0.03792238235473633
[2022/03/02 17:16:15] root DEBUG: cls num  : 1, elapse : 0.004909992218017578
[2022/03/02 17:16:15] root DEBUG: rec_res num  : 1, elapse : 0.003943204879760742
[Text Detection Completed in 0.050 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/03/02 17:16:16] root DEBUG: dt_boxes num : 3, elapse : 0.11153817176818848
[2022/03/02 17:16:16] root DEBUG: cls num  : 3, elapse : 0.011598348617553711
[2022/03/02 17:16:16] root DEBUG: rec_res num  : 3, elapse : 0.03538179397583008
[Text Detection Completed in 0.175 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/03/02 17:16:16] root DEBUG: dt_boxes num : 1, elapse : 0.03396034240722656
[2022/03/02 17:16:16] root DEBUG: cls num  : 1, elapse : 0.00433349609375
[2022/03/02 17:16:16] root DEBUG: rec_res num  : 1, elapse : 0.0049898624420166016
[Text Detection Completed in 0.050 s] Input: data/operation/target.pn

[2022/03/02 17:16:17] root DEBUG: cls num  : 0, elapse : 0
[2022/03/02 17:16:17] root DEBUG: rec_res num  : 0, elapse : 7.152557373046875e-07
[Text Detection Completed in 0.023 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/03/02 17:16:17] root DEBUG: dt_boxes num : 0, elapse : 0.010701894760131836
[2022/03/02 17:16:17] root DEBUG: cls num  : 0, elapse : 0
[2022/03/02 17:16:17] root DEBUG: rec_res num  : 0, elapse : 9.5367431640625e-07
[Text Detection Completed in 0.013 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/03/02 17:16:17] root DEBUG: dt_boxes num : 3, elapse : 0.09698700904846191
[2022/03/02 17:16:17] root DEBUG: cls num  : 3, elapse : 0.011121511459350586
[2022/03/02 17:16:17] root DEBUG: rec_res num  : 3, elapse : 0.034821271896362305
[Text Detection Completed in 0.158 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/03/02 17:16:17] root DEBUG: dt_boxes num : 1, elapse : 0.03360819816589

In [8]:
# ********************************************
# *** Streaming Detection with Key Control ***
# ********************************************
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 960)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 540)

opr_id = 0

while(1):
    ret, frame = cap.read()
    frame = cv2.rotate(frame, cv2.cv2.ROTATE_90_CLOCKWISE)
    frame = frame[80:900, :]
    
    cv2.imshow('camera', frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    elif key == ord('s'):
        cv2.destroyWindow('element matching')
        opr = operations[opr_id]
        opr.show_target_ele(show=True)
        cv2.imwrite('data/output/temp.jpg', frame)
        
        # 1. detect GUI element in the frame         
        gui = GUI('data/output/temp.jpg')
        gui.detect_element(True, True, True, paddle_cor=paddle_ocr)
        
        # 2. focus on the screen region and adjust all elements
        gui.adjust_elements_by_screen()
        matched_ele = gui.match_elements(target_ele_img=opr.target_element_img, target_ele_text=opr.get_text_content(), resnet_model=resnet_model, min_similarity_img=0.55)
        
        # 3. show the result
        if matched_ele is not None:
            board = gui.img.copy()
            matched_ele.draw_element(board, show=False)
            board = cv2.hconcat([opr.show_target_ele(show=False), board, gui.det_result_imgs['merge']])
            cv2.imshow('element matching', board)
            opr_id += 1
        print('===================\n')
cap.release()
cv2.destroyAllWindows()

[2022/03/02 17:17:56] root DEBUG: dt_boxes num : 16, elapse : 0.13343477249145508
[2022/03/02 17:17:56] root DEBUG: cls num  : 16, elapse : 0.05554795265197754
[2022/03/02 17:17:56] root DEBUG: rec_res num  : 16, elapse : 0.10076284408569336
[Text Detection Completed in 0.306 s] Input: data/output/temp.jpg Output: data/output/ocr/temp.json
[Compo Detection Completed in 0.421 s] Input: data/output/temp.jpg Output: data/output/ip/temp.json
[Merge Completed] Input: data/output/temp.jpg Output: data/output/merge/temp.jpg
0.30547148
0.5700277
0.33187753
0.38271827
0.33264926
0.32875717
0.2819058
0.31525716
0.20916313
Match by image

[2022/03/02 17:18:09] root DEBUG: dt_boxes num : 13, elapse : 0.15924644470214844
[2022/03/02 17:18:09] root DEBUG: cls num  : 13, elapse : 0.04620552062988281
[2022/03/02 17:18:09] root DEBUG: rec_res num  : 13, elapse : 0.09777522087097168
[Text Detection Completed in 0.321 s] Input: data/output/temp.jpg Output: data/output/ocr/temp.json
[Compo Detection Compl

In [9]:
gui.show_detection_result()

In [8]:
operations[3].show_target_ele()
operations[3].target_element_text

[]

In [14]:
gui.match_elements(target_ele_img=operations[3].target_element_img, resnet_model=resnet_model, show=True)

No matched element found


In [4]:
# ***************************
# *** Streaming Detection ***
# ***************************
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 960)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 540)

while(1):
    ret, frame = cap.read()
    frame = cv2.rotate(frame, cv2.cv2.ROTATE_90_CLOCKWISE)
    frame = frame[80:900, :]
    
#     cv2.imshow('camera', frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    cv2.imwrite('data/output/temp.jpg', frame)
    gui = GUI('data/output/temp.jpg')
    gui.detect_element(True, True, True, paddle_cor=paddle_ocr)
    gui.recognize_phone_screen()
    cv2.imshow('camera', gui.det_result_imgs['merge'])
    cv2.imshow('screen', gui.draw_screen(show=False))
    print('===================\n')
cap.release()
cv2.destroyAllWindows()

[2022/03/01 21:55:58] root DEBUG: dt_boxes num : 4, elapse : 0.18249082565307617
[2022/03/01 21:55:58] root DEBUG: cls num  : 4, elapse : 0.01497507095336914
[2022/03/01 21:55:58] root DEBUG: rec_res num  : 4, elapse : 0.04303288459777832
[Text Detection Completed in 0.255 s] Input: data/output/temp.jpg Output: data/output/ocr/temp.json
[Compo Detection Completed in 0.483 s] Input: data/output/temp.jpg Output: data/output/ip/temp.json
[Merge Completed] Input: data/output/temp.jpg Output: data/output/merge/temp.jpg

[2022/03/01 21:55:59] root DEBUG: dt_boxes num : 13, elapse : 0.14171838760375977
[2022/03/01 21:55:59] root DEBUG: cls num  : 13, elapse : 0.04781770706176758
[2022/03/01 21:55:59] root DEBUG: rec_res num  : 13, elapse : 0.08809828758239746
[Text Detection Completed in 0.294 s] Input: data/output/temp.jpg Output: data/output/ocr/temp.json
[Compo Detection Completed in 0.489 s] Input: data/output/temp.jpg Output: data/output/ip/temp.json
[Merge Completed] Input: data/output/

In [1]:
# *******************************
# *** Adjust Video Clip Range ***
# *******************************

import cv2

def nothing(x):
    pass

cv2.namedWindow('win')
cv2.createTrackbar('top', 'win', 80, 960, nothing)
cv2.createTrackbar('left', 'win', 0, 540, nothing)
cv2.createTrackbar('bottom', 'win', 900, 960, nothing)
cv2.createTrackbar('right', 'win', 540, 540, nothing)

cap = cv2.VideoCapture(1, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 960)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 540)
print('Camera Loaded')

while(1):
    ret, frame = cap.read()
    frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    
    top = cv2.getTrackbarPos('top', 'win')
    left = cv2.getTrackbarPos('left', 'win')
    bottom = cv2.getTrackbarPos('bottom', 'win')
    right = cv2.getTrackbarPos('right', 'win')
    
    frame_clip = frame[top:bottom, left:right]
    cv2.imshow('frame', frame)
    cv2.imshow('clip', frame_clip)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Camera Loaded


In [3]:
cap.read()

(False, None)